# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

### Import All Required Dependencies

In [1]:
import logging
import os
import csv
import shutil


from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

#Computer target
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException


#Visualizing run
from azureml.widgets import RunDetails

#library for Saving model
import joblib

#import clean data function defined in train.py script
from train import clean_data 

#ONNX libraries
import sys
import json
from azureml.automl.core.onnx_convert import OnnxConvertConstants
from azureml.train.automl import constants

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.20.0


### Initialize Workspace
Initialize a workspace object from persisted configuration. Make sure the config file is present at .\config.json

In [2]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

quick-starts-ws-136079
aml-quickstarts-136079
southcentralus
510b94ba-e453-4417-988b-fbdc37b55ca7


### Setup Experiment 
Here we will be creating an experiment named "heart-disease-automl".  

In [3]:
# choose a name for experiment
experiment_name = 'liver-disease-automl'

experiment=Experiment(ws, experiment_name)

### Create or Attach an AmlCompute cluster
You will need to create a compute target for your AutoML run. In this demo, you get the default AmlCompute as your training compute resource.

In [4]:
#Create compute cluster

# Choose a name for your CPU cluster
cpu_cluster_name = "notebook136079"
# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS3_V2',
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

Found existing cluster, use it.

Running


## Dataset

TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.
TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

### Overview
There is an increasing number of patients with liver disease in recent time due to life style and living habits such as excessive alcohol consumption, inhale of harmful gases, excessive weight gain, intake of contaminated food, abuse of drugs. This dataset is aimed at helping doctors during clinical diagnosis of liver disease to elevate burden and the stress involved in analyzing every single patients’ information. Therefore, the goal is to create a classifier that predicts whether a subject is healthy (non-liver patient) or ill (liver patient) based on some clinical and demographic features which are: age, gender, total Bilirubin, direct Bilirubin, total proteins, albumin, A/G ratio, SGPT, SGOT and Alkphos.

In [7]:
#load the liver dataset to datastore
data_path = "https://raw.githubusercontent.com/chollette/nd00333-capstone/master/Liver%20Patient%20Dataset%20(LPD)_train.csv"
dataset = pd.read_csv(data_path)

In [9]:
# Use the clean_data function to clean your data.
x, y = clean_data(dataset)
train_data = pd.concat([x, y], axis=1, sort=False)
#upload the cleaned marketing data to the default datastore (blob) of my workspace.

#first convert data to .csv
train_data.to_csv('train_data.csv',header=True)

#Then upload to datastore
datastore = ws.get_default_datastore()
datastore.upload_files(['train_data.csv'], target_path='', overwrite=True)

Uploading an estimated of 1 files
Uploading train_data.csv
Uploaded train_data.csv, 1 files out of an estimated total of 1
Uploaded 1 files


$AZUREML_DATAREFERENCE_workspaceblobstore

In [11]:
#convert back to tabular dataset for running in AutoML
train_data = Dataset.Tabular.from_delimited_files(path = [(datastore, 'train_data.csv')])
label = "Result"

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [12]:
# TODO: Put your automl settings here
automl_settings = {   
    "experiment_timeout_hours": 1,
    #"experiment_timeout_minutes": 30,
    "enable_early_stopping" : True,
    "model_explainability" : True,
    "iteration_timeout_minutes": 5,
    "max_concurrent_iterations": 5,
    "max_cores_per_iteration": -1,
    "n_cross_validations": 10,
    "primary_metric": 'accuracy',
    "featurization": 'auto',
    "verbosity": logging.INFO,
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(task = 'classification',
                             compute_target=compute_target,
                             training_data = train_data,
                             label_column_name = label,
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

In [13]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)

Running on remote.


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [14]:
#Visualize experiment
RunDetails(remote_run).show()


_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [15]:
remote_run.wait_for_completion()

{'runId': 'AutoML_4c1de892-2ee5-447c-889a-9ed6addd99e3',
 'target': 'notebook136079',
 'status': 'Completed',
 'startTimeUtc': '2021-01-26T06:36:03.480482Z',
 'endTimeUtc': '2021-01-26T06:55:32.205617Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '10',
  'target': 'notebook136079',
  'AMLSettingsJsonString': '{"path":null,"name":"liver-disease-automl","subscription_id":"510b94ba-e453-4417-988b-fbdc37b55ca7","resource_group":"aml-quickstarts-136079","workspace_name":"quick-starts-ws-136079","region":"southcentralus","compute_target":"notebook136079","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":1000,"primary_metric":"accuracy","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":10,"y_min":null,"y_max":null,"nu

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [16]:
# Retrieve and save your best automl model.
best_automl_run_metrics = remote_run.get_metrics()
print(best_automl_run_metrics)

{'experiment_status': ['DatasetEvaluation', 'FeaturesGeneration', 'DatasetFeaturization', 'DatasetFeaturizationCompleted', 'DatasetCrossValidationSplit', 'ModelSelection'], 'experiment_status_description': ['Gathering dataset statistics.', 'Generating features for the dataset.', 'Beginning to fit featurizers and featurize the dataset.', 'Completed fit featurizers and featurizing the dataset.', 'Generating individually featurized CV splits.', 'Beginning model selection.'], 'f1_score_micro': 0.9997790597699465, 'norm_macro_recall': 0.999221928809788, 'AUC_weighted': 0.9997547986558141, 'recall_score_weighted': 0.9997790597699465, 'average_precision_score_micro': 0.9998045339859013, 'log_loss': 0.21306852431045414, 'balanced_accuracy': 0.9996109644048939, 'average_precision_score_weighted': 0.9998234797972634, 'AUC_macro': 0.9997547986558141, 'precision_score_weighted': 0.9997792113992109, 'recall_score_macro': 0.9996109644048939, 'recall_score_micro': 0.9997790597699465, 'accuracy': 0.99

In [17]:
print("Best AutoML model Accuracy: ", best_automl_run_metrics['accuracy'])

Best AutoML model Accuracy:  0.9997790597699465


In [18]:
#TODO: Save the best model
best_run, fitted_model = remote_run.get_output()
print(best_run)

Run(Experiment: liver-disease-automl,
Id: AutoML_4c1de892-2ee5-447c-889a-9ed6addd99e3_38,
Type: azureml.scriptrun,
Status: Completed)


In [19]:
joblib.dump(fitted_model, 'automl-votingEnsemble_model.joblib')

['automl-votingEnsemble_model.joblib']

In [21]:
compute_target.delete()